In [4]:
# Este notebook tiene como objetivo hacer un control de calidad de  los datos de desastres naturales
  # descargados desde el portal de datos abiertos de la UNGRD


In [1]:
# #importamos las librerías a utilizar
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.patches import Rectangle
# import matplotlib.patches as mpatches
# import seaborn as sns 
# import datetime
# import calendar
# import matplotlib.dates as mdates
# import re
# from difflib import SequenceMatcher
# from unidecode import unidecode
# import unicodedata

In [2]:
#Importamos archivo de emergencias consolidadas de la UNGRD (2000-2023) para procesar info de desastres
#df = pd.read_csv("Emergencias_puras_procesadas.csv", encoding='latin1')

In [3]:
# Separamos los municipios que están en una misma celda en filas diferentes
         #Usar expresión regular para separar por "-" o "/" o "- "
#df['MUNICIPIO'] = df['MUNICIPIO'].str.split(r'\s*[-/]\s*')
# Explode para separar los municipios
#df = df.explode('MUNICIPIO')


In [4]:
#Creamos una función para eliminar los espacios antes y después del nombre en las columnas deseadas
 #Utilizamos el método strip para hacer esto

# Seleccionar las columnas a las que se quiere aplicar el strip
#cols_strip = ['DEPARTAMENTO', 'MUNICIPIO']

# Aplicar str.strip() a las columnas seleccionadas
#df[cols_strip] = df[cols_strip].apply(lambda x: x.str.strip())


In [7]:

#Utilizamos una función para encontrar municipios que se parezcan en un 80% en su escritura para encontrar posibles errores
 #Si se encuentran municipios con más de 80% de coincidencia en un departamento se le pregunta al usuario si ambos municipios...
    #son correctos, y si no lo son, se le pregunta cuál municipio quiere dejar, si A o B.
    

## Función para encontrar municipios similares
#def encontrar_similares(df, threshold=0.8):
#    municipios_corregidos = {}
#    for departamento in df['DEPARTAMENTO'].unique():
#        municipios = df[df['DEPARTAMENTO'] == departamento]['MUNICIPIO'].unique()
#        for i, municipio1 in enumerate(municipios):
#            for municipio2 in municipios[i+1:]:
#                ratio = SequenceMatcher(None, municipio1, municipio2).ratio()
#                if ratio >= threshold:
#                    print(f"Departamento: {departamento}")
#                    print(f"Municipios similares: *{municipio1}* - *{municipio2}* (similitud: {ratio:.2f})")
#                    
#                    # Pregunta si ambos municipios están correctos
#                    respuesta = input("¿Ambos nombres son correctos? (si/no): ").lower()
#                    
#                    if respuesta == 'no':
#                        correcto = input(f"¿Cuál es el nombre correcto? (1 para '{municipio1}', 2 para '{municipio2}'): ")
#                        if correcto == '1':
#                            municipios_corregidos[municipio2] = municipio1
#                        elif correcto == '2':
#                            municipios_corregidos[municipio1] = municipio2
#    
#    return municipios_corregidos
#

## Función para reemplazar los municipios incorrectos por los correctos
#def corregir_municipios(df, municipios_corregidos):
#    df['MUNICIPIO'] = df['MUNICIPIO'].replace(municipios_corregidos)
#    return df
#
## Encuentra los municipios similares
#municipios_corregidos = encontrar_similares(df)
#
## Aplica las correcciones al DataFrame
#df_corregido = corregir_municipios(df, municipios_corregidos)
#



In [8]:
#exportamos a excel para verificar en profundidad
#df.to_excel('C:/Users/migue/Proyecto_Col/Rtado_prueba/df_reemplazo_1.xlsx',index=False)

In [10]:
 #Se trabaja con el nuevo dataframe para terminar de hacer correciones
    #El nuevo archivo tiene las correcciones de todo los pasos anteriores.

##Se invoca el dataframe que se generó y corregió por medio de excel en el paso anterior
# #Importamos el archivo del primer dataframe procesado
#df2 = pd.read_csv("Reemplazo1_rev.csv", encoding='latin1')
#df2

In [11]:
##Hay separaciones de municipios que no se hicieron porque están separadas por comas y no se incluyo en el primer filtro
##Este código separa los municipios que faltaron por separar incluyendo los que se separan por comas, ejm:
#   #Cartagena, carmen, san jacinto...
#    
## Separamos los municipios que están en una misma celda en filas diferentes
#         #Usar expresión regular para separar por "," 
#     #PERO APLICANDO LA EXCEPCIÓN DE BOGOTÁ.
#        
## Separar las filas que tienen "BOGOTÁ, D.C." para no dividirlas
#df_bogota = df2[df2['MUNICIPIO'] == 'BOGOTÁ, D.C.']
#
## Filtrar el resto del DataFrame que no contiene "BOGOTÁ, D.C."
#df_otros = df2[df2['MUNICIPIO'] != 'BOGOTÁ, D.C.']
#
## Aplicar el split y explode a las filas que no contienen "BOGOTÁ, D.C."
#df_otros['MUNICIPIO'] = df_otros['MUNICIPIO'].str.split(r'\s*[,]\s*')
#df_otros = df_otros.explode('MUNICIPIO')
#
## Unir nuevamente las filas con "BOGOTÁ, D.C." con el resto del DataFrame
#df2 = pd.concat([df_bogota, df_otros], ignore_index=True)
#
#

In [12]:
# #Ahora para no tener que exportar todo el dataframe a excel, creo un código para ver los municipios que están en un departamento
#    #Ver municipios en un departamento para ver si funciona el código anterior 
#    
## Filtrar los municipios de un departamento específico
##depto = "CUNDINAMARCA"  # Cambia esto por el departamento de interes
#mpios = df2[df2['DEPARTAMENTO'] == depto]['MUNICIPIO'].unique()
#
## Mostrar los municipios
##print(f"Municipios en el departamento de {depto}:")
##for mpio in mpios:
##    print(mpio)

In [13]:
##Ahora voy a comparar los nombres de los departamentos y municipios que tengo en el dataframe con los datos oficiales
# #Para eso uso un archivo de la base de datos abiertos de Colombia
#
##El archivo tiene caracteres especiales, por eso primero lo voy a procesar
#   #El archivo va a quedar con tildes, sin caracteres especiales y en mayuscula
#    
## Cargar el archivo CSV (asegúrate de usar la codificación correcta)
#df_oficial = pd.read_csv("Departamentos_Municipios_Colombia.csv")
#
##Este código tranforma las tildes en formato UTF-8 a su forma normalizada (AmagÃ¡" se convertirá a "AMAGÁ")
#def corregir_nombre(valor):
#    if isinstance(valor, str):  # Verifica si el valor es una cadena de texto
#        # Normalizar la cadena a su forma compuesta y luego convertir a mayúsculas
#        valor = unicodedata.normalize('NFC', valor).upper()
#    return valor  # Retorna el valor original si no es una cadena de texto
#
## Aplicar la corrección a todas las columnas del DataFrame de datos oficiales 
#df_oficial = df_oficial.applymap(corregir_nombre)
#
##Aplicar la correción a todas las columnas del dataframde de datos de desastres
#df2=df2.applymap(corregir_nombre)

In [14]:
#SE HACE UNA VERIFICACIÓN PARA VER LAS DIFERENCIAS ENTRE LOS DEPARTAMENTOS en df2 y df_oficial

In [15]:
##Según la verificación anterior, voy a reemplazar el nombre de "BOGOTA, D.C." por "BOGOTÁ D.C." para unificar los nombres.
# #Se reemplaza "BOGOTA, D.C." por "BOGOTÁ D.C." en df2 
#    
#df2['DEPARTAMENTO'] = df2['DEPARTAMENTO'].replace('BOGOTA, D.C.', 'BOGOTÁ D.C.')
#

In [16]:
##Según la verificación de las diferencias entre dataframes, creamos código para reemplazar los nombres de los departamentos...
# #...en el df de desastres que no tienen tilde respecto al de los datos abiertos (oficiales).
#    
#
## Asegurarte de que ambas columnas de DEPARTAMENTO estén en mayúsculas y normalizadas para comparación
#df2['DEPARTAMENTO'] = df2['DEPARTAMENTO'].str.upper()
#df_oficial['DEPARTAMENTO'] = df_oficial['DEPARTAMENTO'].str.upper()
#
#def normalizar(texto):
#    return unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')
#
## Crear un diccionario para mapear los nombres de departamentos en df2 a los correctos en df_oficial
#departamentos_oficiales = {normalizar(depto): depto for depto in df_oficial['DEPARTAMENTO']}
#
## Reemplazar los nombres en df2 por los correctos en df_oficial
#df2['DEPARTAMENTO'] = df2['DEPARTAMENTO'].apply(lambda x: departamentos_oficiales.get(normalizar(x), x))
#

In [17]:
##Ahora voy a comparar los datos de departamentos y municipios del df de desastres (df2) respecto al df de datos abiertos (df_oficial)
# #para ver si los deptos y mpios sí coinciden 
#    
##Primero verifico los departamentos:
## Obtener los departamentos únicos en ambos DataFrames
#departamentos_desastres = set(df2['DEPARTAMENTO'].unique())
#departamentos_oficiales = set(df_oficial['DEPARTAMENTO'].unique())
#
## Encontrar los departamentos que están en desastres pero no en el oficial
#departamentos_diferentes_desastres = departamentos_desastres - departamentos_oficiales
#print(f"Departamentos en el DataFrame de desastres que no están en el oficial: {departamentos_diferentes_desastres}")
#
## Encontrar los departamentos que están en el oficial pero no en desastres
#departamentos_diferentes_oficiales = departamentos_oficiales - departamentos_desastres
#print(f"Departamentos en el DataFrame oficial que no están en desastres: {departamentos_diferentes_oficiales}")
#

In [18]:
##En este código voy a unificar los nombres de los municipios según su nombre oficial (de acuerdo a datos abiertos del gobierno nal)
# #Estandarizo el nombre de los municipios para que sean iguales en ambos dataframes
#    #Utilizo la función normalizar usada anteriormente
#    
# # que ambas columnas de MUNICIPIO estén en mayúsculas para comparación
#df2['MUNICIPIO'] = df2['MUNICIPIO'].str.upper()
#df_oficial['MUNICIPIO'] = df_oficial['MUNICIPIO'].str.upper()
#
## Iterar sobre cada departamento en df2
#for departamento in df2['DEPARTAMENTO'].unique():
#    # Crear un diccionario para mapear los nombres de municipios en df2 a los correctos en df_oficial
#    municipios_oficiales = {normalizar(mun): mun for mun in df_oficial[df_oficial['DEPARTAMENTO'] == departamento]['MUNICIPIO']}
#    
#    # Reemplazar los nombres en df2 por los correctos en df_oficial dentro del mismo departamento
#    df2.loc[df2['DEPARTAMENTO'] == departamento, 'MUNICIPIO'] = df2[df2['DEPARTAMENTO'] == departamento]['MUNICIPIO'].apply(lambda x: municipios_oficiales.get(normalizar(x), x))
#
#

In [19]:
##EN ESTE CÓDIGO COMPARO LOS MUNICIPIOS QUE ESTÁN PRESENTES EN EL DF2 y no en el oficial y viceversa.
#
## Obtener los departamentos únicos en ambos DataFrames
#departamentos_desastres = set(df2['DEPARTAMENTO'].unique())
#departamentos_oficiales = set(df_oficial['DEPARTAMENTO'].unique())
#
## Encontrar los departamentos que están en desastres pero no en el oficial
#departamentos_diferentes_desastres = departamentos_desastres - departamentos_oficiales
#print(f"Departamentos en el DataFrame de desastres que no están en el oficial: {departamentos_diferentes_desastres}")
#
## Encontrar los departamentos que están en el oficial pero no en desastres
#departamentos_diferentes_oficiales = departamentos_oficiales - departamentos_desastres
#print(f"Departamentos en el DataFrame oficial que no están en desastres: {departamentos_diferentes_oficiales}")
#
#
## Crear un diccionario para almacenar las diferencias en municipios por departamento
#municipios_diferencias = {}
#
## Iterar sobre cada departamento del DataFrame oficial
#for departamento in departamentos_oficiales:
#    # Obtener los municipios únicos en el DataFrame de desastres para este departamento
#    municipios_desastres = set(df2[df2['DEPARTAMENTO'] == departamento]['MUNICIPIO'].unique())
#    
#    # Obtener los municipios únicos en el DataFrame oficial para este departamento
#    municipios_oficiales = set(df_oficial[df_oficial['DEPARTAMENTO'] == departamento]['MUNICIPIO'].unique())
#    
#    # Encontrar las diferencias
#    municipios_diferentes_desastres = municipios_desastres - municipios_oficiales
#    municipios_diferentes_oficiales = municipios_oficiales - municipios_desastres
#    
#    # Almacenar las diferencias si existen
#    if municipios_diferentes_desastres or municipios_diferentes_oficiales:
#        municipios_diferencias[departamento] = {
#            "Solo en desastres": municipios_diferentes_desastres,
#            "Solo en oficial": municipios_diferentes_oficiales
#        }
#
## Mostrar las diferencias encontradas
#for departamento, diferencias in municipios_diferencias.items():
#    print(f"\nDepartamento: {departamento}")
#    print(f"Municipios solo en desastres: {diferencias['Solo en desastres']}")
#    print(f"Municipios solo en oficial: {diferencias['Solo en oficial']}")
#

In [20]:
##Exporto ambos dataframes para tratar manualmente las diferencias
#
#df2.to_excel('C:/Users/migue/Proyecto_Col/Rtado_prueba/df_reemplazo_2.xlsx',index=False)
#df_oficial.to_excel('C:/Users/migue/Proyecto_Col/Rtado_prueba/df_oficial_procesado.xlsx',index=False)

In [22]:
##Ahora como control de calidad de los datos se van a eliminar las celdas que tengan como municipio = "DEPARTAMENTO" o "VARIOS"
#    #también las que tengan celdas vacías o departamento=NACION
##Importamos el nuevo dataframe llamado Revision2.csv y lo nombramos df3
#
#df3=pd.read_csv("Rtado_prueba/Revision2_nvo.csv", encoding='latin1')


In [23]:
##Ahora procedo a eliminar las celdas vacías de la columna "DEPARTAMENTO" y los registros que tengan "NACION" como departamento"
# #También elimino las filas donde el departamento esté vacío
#
## Eliminar filas donde el departamento es "NACION"
#df3 = df3[df3['DEPARTAMENTO'] != 'NACION']
#
## Eliminar filas donde la columna "DEPARTAMENTO" tiene valores vacíos o NaN
#df3 = df3.dropna(subset=['DEPARTAMENTO'])
#
## O también puedes eliminar filas donde la columna "DEPARTAMENTO" está vacía
#df3 = df3[df3['DEPARTAMENTO'].str.strip() != '']
#

In [24]:
##Ahora elimino las filas que tengan como municipio=DEPARTAMENTO (control de calidad de datos)
# #También las que tengan columnas de municipio=VARIOS
#
#df3 = df3[df3['MUNICIPIO'] != 'DEPARTAMENTO']
#df3 = df3[df3['MUNICIPIO'] != 'VARIOS']
#

In [25]:
##Se carga archivo del DANE para observar diferencias en departamentos y municipios, luego se añaden los códigos del DANE
#
#
##Se comparan los departamentos primero
## Cargar el archivo CSV (asegúrate de usar la codificación correcta)
#df_oficial = pd.read_csv("Rtado_prueba/df_oficial_procesado.csv", encoding='latin1')
#    
## Obtener los departamentos únicos en ambos DataFrames
#departamentos_desastres = set(df3['DEPARTAMENTO'].unique())
#departamentos_oficiales = set(df_oficial['DEPARTAMENTO'].unique())
#
## Encontrar los departamentos que están en desastres pero no en el oficial
#departamentos_diferentes_desastres = departamentos_desastres - departamentos_oficiales
#print(f"Departamentos en el DataFrame de desastres que no están en el oficial: {departamentos_diferentes_desastres}")
#
## Encontrar los departamentos que están en el oficial pero no en desastres
#departamentos_diferentes_oficiales = departamentos_oficiales - departamentos_desastres
#print(f"Departamentos en el DataFrame oficial que no están en desastres: {departamentos_diferentes_oficiales}")


In [26]:
##Ahora se comparan los municipios para observar diferencias
## Crear un diccionario para almacenar las diferencias en municipios por departamento
#municipios_diferencias = {}
#
## Iterar sobre cada departamento del DataFrame oficial
#for departamento in departamentos_oficiales:
#    # Obtener los municipios únicos en el DataFrame de desastres para este departamento
#    municipios_desastres = set(df3[df3['DEPARTAMENTO'] == departamento]['MUNICIPIO'].unique())
#    
#    # Obtener los municipios únicos en el DataFrame oficial para este departamento
#    municipios_oficiales = set(df_oficial[df_oficial['DEPARTAMENTO'] == departamento]['MUNICIPIO'].unique())
#    
#    # Encontrar las diferencias
#    municipios_diferentes_desastres = municipios_desastres - municipios_oficiales
#    municipios_diferentes_oficiales = municipios_oficiales - municipios_desastres
#    
#    # Almacenar las diferencias si existen
#    if municipios_diferentes_desastres or municipios_diferentes_oficiales:
#        municipios_diferencias[departamento] = {
#            "Solo en desastres": municipios_diferentes_desastres,
#            "Solo en oficial": municipios_diferentes_oficiales
#        }
#
## Mostrar las diferencias encontradas
#for departamento, diferencias in municipios_diferencias.items():
#    print(f"\nDepartamento: {departamento}")
#    print(f"Municipios solo en desastres: {diferencias['Solo en desastres']}")
#    print(f"Municipios solo en oficial: {diferencias['Solo en oficial']}")
#    
#

In [27]:
#df3

In [28]:
##Ahora elimino las filas que tengan eso en MUNICIPIO= "GOBERNACIÓN",'OCÉANO PACÍFICO', o nan
#
## Eliminar filas donde el municipio es "GOBERNACIÓN", "OCÉANO PACÍFICO" o NaN
#df3 = df3[~df3['MUNICIPIO'].isin(['GOBERNACIÓN', 'OCÉANO PACÍFICO','VAUPES'])]
#
## Eliminar filas donde el municipio es NaN
#df3 = df3.dropna(subset=['MUNICIPIO'])
#df3

In [29]:
##Elimino las columnas sin nombre y vacías
#
#df3 = df3.drop(columns=['Unnamed: 12','Unnamed: 13'])

In [30]:
##Ahora agrego las columnas de los códigos del DANE como identificadores a df3, para departamentos y municipios.
# #Esto para luego tener formas de identificar los municipios de manera más fácil
#    
#
## Crear una columna combinada de "DEPARTAMENTO" y "MUNICIPIO" en df_oficial
#df_oficial['DEP_MUN'] = df_oficial['DEPARTAMENTO'] + "_" + df_oficial['MUNICIPIO']
#
## Crear una columna combinada de "DEPARTAMENTO" y "MUNICIPIO" en df3
#df3['DEP_MUN'] = df3['DEPARTAMENTO'] + "_" + df3['MUNICIPIO']
#
## Crear un diccionario que mapea la columna combinada a "CÓDIGO DANE DEL MUNICIPIO"
#codigo_dane_map = df_oficial.set_index('DEP_MUN')['CÓDIGO DANE DEL MUNICIPIO'].to_dict()
#
## Crear la nueva columna "CÓDIGO DANE" en df3 usando el diccionario de mapeo
#df3['CÓDIGO DANE'] = df3['DEP_MUN'].map(codigo_dane_map)
#
## Eliminar la columna temporal 'DEP_MUN' si ya no es necesaria
#df3.drop(columns=['DEP_MUN'], inplace=True)


In [31]:
##Como los municipios que están diferente en df3 respecto a los nombres oficiales en el DANE se verificaron uno por uno
# #con el nombre de su sitio oficial y en df3 está el correcto, entonces se hacen las equivalencias respectivas 
#    #con el código del municipio en el df_oficial (DANE) pero con el nombre de df3 que es el oficial de su pag web
#     #o sea, se agrega al df3 la columna de código DANE del mpio pero con el nombre df3
#        
## Diccionario de equivalencias municipio en df3 : municipio en df_oficial
#equivalencias_municipios = {
#    # CAUCA
#    'LOPEZ DE MICAY': 'LÓPEZ',
#    # MAGDALENA
#    'PUEBLOVIEJO': 'PUEBLO VIEJO',
#    'CERRO DE SAN ANTONIO': 'CERRO SAN ANTONIO',
#    # SUCRE
#    'TOLUVIEJO': 'TOLÚ VIEJO',
#    # META
#    'VISTAHERMOSA': 'VISTA HERMOSA',
#    # HUILA
#    'EL AGRADO': 'AGRADO',
#    'EL PITAL': 'PITAL',
#    
#    # CUNDINAMARCA
#    'SAN JUAN DE RIOSECO': 'SAN JUAN DE RÍO SECO',
#    'UBATE': 'VILLA DE SAN DIEGO DE UBATE',
#    # PUTUMAYO
#    'PUERTO LEGUÍZAMO': 'LEGUÍZAMO',
#    'VALLE DEL GUAMUEZ': 'VALLE DE GUAMEZ',
#    # BOLÍVAR
#    'MOMPOX': 'MOMPÓS',
#    'SAN PABLO': 'SAN PABLO DE BORBUR',
#    # ANTIOQUIA
#    'SANTA FE DE ANTIOQUIA': 'SANTAFÉ DE ANTIOQUIA',
#    'SAN VICENTE FERRER': 'SAN VICENTE',
#    'EL RETIRO': 'RETIRO',
#    'SAN PEDRO DE LOS MILAGROS': 'SAN PEDRO',
#    'EL PEÑOL': 'PEÑOL',
#    # LA GUAJIRA
#    'DIBULLA': 'DIBULA',
#    # NARIÑO
#    'MAGUI PAYAN': 'MAGÜÍ',
#    'EL CONTADERO': 'CONTADERO',
#    # GUAINÍA
#    'BARRANCOMINAS': 'BARRANCO MINAS',
#    # CÓRDOBA
#    'SAN ANDRÉS DE SOTAVENTO': 'SAN ANDRÉS SOTAVENTO',
#    # TOLIMA
#    'EL ESPINAL': 'ESPINAL',
#    'RIOBLANCO': 'RIO BLANCO',
#    # BOYACÁ
#    'BUENAVISTA': 'BUENA VISTA'
#}
#
#
## Crear una copia del df_oficial con las columnas relevantes
#df_oficial_reducido = df_oficial[['DEPARTAMENTO', 'MUNICIPIO', 'CÓDIGO DANE DEL MUNICIPIO']].copy()
#
## Reemplazar los nombres de los municipios en df3 según las equivalencias para la comparación
#df3['MUNICIPIO_EQUIVALENTE'] = df3['MUNICIPIO'].replace(equivalencias_municipios)
#
## Realizar la unión (merge) usando las equivalencias en la columna "MUNICIPIO_EQUIVALENTE"
#df3 = df3.merge(df_oficial_reducido, 
#                left_on=['DEPARTAMENTO', 'MUNICIPIO_EQUIVALENTE'], 
#                right_on=['DEPARTAMENTO', 'MUNICIPIO'], 
#                how='left', 
#                suffixes=('', '_OFICIAL'))
#
## Eliminar la columna temporal "MUNICIPIO_EQUIVALENTE" y la columna "MUNICIPIO_OFICIAL"
#df3 = df3.drop(columns=['MUNICIPIO_EQUIVALENTE', 'MUNICIPIO_OFICIAL'])


In [32]:
##Hay un error en el df oficial respeto a SAN LUIS DE PALENQUE  en CASANARE, lo tienen como SAN LUIS DE GACENO
# #Se pone el código oficial del municipio en la columna CÓDIGO DANE DEL MUNICIPIO_OFICIAL
#    
##También SANTA CATALINA no figura como municipio entonces lo elimino, además de solo tener un registro
#
#
## Condición para encontrar las filas donde el municipio es "SAN LUIS DE PALENQUE"
#condicion = (df3['MUNICIPIO'] == 'SAN LUIS DE PALENQUE') & (df3['DEPARTAMENTO'] == 'CASANARE')
#
## Asignar el código DANE correcto como float
#df3.loc[condicion, 'CÓDIGO DANE'] = float('85.325')
#
##Ahora elimino a SANTA CATALINA EN el depto de san andrés
## Crear la condición para las filas a eliminar
#condicion_eliminar = (df3['MUNICIPIO'] == 'SANTA CATALINA') & (df3['DEPARTAMENTO'] == 'ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA')
#
## Eliminar las filas que cumplen con la condición
#df3 = df3[~condicion_eliminar]

In [33]:
##Por último, elimino la columna CÓDIGO DANE porque la columna CÓDIGO DANE DEL MUNICIPIO contiene todos los códigos
#
## Eliminar la columna "CÓDIGO DANE DEL MUNICIPIO"
#df3 = df3.drop(columns=['CÓDIGO DANE'])

In [34]:
#GUARDAMOS ESTE PROCESAMIENTO como df_mundep

#df3.to_csv('Rtado_prueba/df_mundep.csv',index=False)
#df3.to_excel('Rtado_prueba/df_mundep.xlsx',index=False)

In [35]:

#  #Ahora voy a procesar los tipos de evento
# df4=pd.read_csv("Rtado_prueba/df_mundep.csv", encoding='UTF-8')

In [36]:
# #primero nos aseguramos de que no hayan espacios antes y después del nombre en tipo de evento, usamos strip
# # Seleccionar las columnas a las que se quiere aplicar el strip
# cols_strip = ['TIPO DE EVENTO']

# # Aplicar str.strip() a las columnas seleccionadas
# df4[cols_strip] = df4[cols_strip].apply(lambda x: x.str.strip())


In [37]:
# #Ahora me aseguro de que se unifique la escritura con y sin tilde (ejm: INUNDACIÓN-INUNDACION)

# from unidecode import unidecode

# # Crear un diccionario para almacenar el mapeo de eventos originales con tilde
# eventos_con_tilde = {}

# # Iterar sobre los eventos únicos en la columna 'TIPO DE EVENTO'
# for evento in df4['TIPO DE EVENTO'].unique():
#     # Convertir el evento a su versión sin tildes
#     evento_sin_tilde = unidecode(evento)
    
#     # Si la versión sin tilde ya está en el diccionario, mantener el evento original con tilde
#     if evento_sin_tilde not in eventos_con_tilde:
#         eventos_con_tilde[evento_sin_tilde] = evento

# # Reemplazar en la columna 'TIPO DE EVENTO' los eventos por su versión con tilde
# df4['TIPO DE EVENTO'] = df4['TIPO DE EVENTO'].apply(lambda x: eventos_con_tilde[unidecode(x)])


In [38]:
# #Ahora elimino los eventos que no corresponden a la categoría o están mal categorizados:

# # Lista de eventos que deseas eliminar
# eventos_a_eliminar = [
#     'RESCATE EN MONTAÑA', 'SEQUIA/INCENDIOS FORESTALES', 'ANTROPICO', 'EPIDEMIA',
#     'VENDAVAL-INCENDIO ESTRUCTURAL', 'EROSION-INCENDIO ESTRUCTURAL', 'INTOXICACION', 'AR',
#     'ANTROPOGENICO NO INTENCIONAL', 'INCENDIO - OTROS', 'CAIDA DE ARBOL', 'DESAPARECIDO',
#     'SOCAVACION', 'EVENTO MAYOR', 'MAREA ALTA', 'CASO FORTUITO', 'IMERSION', 'CONATO',
#     'DIAPIRISMO', 'COVID-19', 'CICLON TROPICAL: DEPRESION/TORMENTA/HURACAN', 'QUEMA', 'DAÑO',
#     'AMENAZAS CONCATENADAS O COMPLEJAS', 'CICLÓN, TROPICAL, DEPRESIÓN/TORMENTA/HURACÁN'
# ]

# # Filtrar el DataFrame para eliminar las filas con los eventos especificados
# df4 = df4[~df4['TIPO DE EVENTO'].isin(eventos_a_eliminar)]



In [39]:
# #Ahora realizo los reemplazos que correspondan según las categorías

# # Diccionario de reemplazos para unificar los desastres naturales
# reemplazos_unificacion = {
#     'AVALANCHA': 'MOVIMIENTO EN MASA',
#     'REMOCION EN MASA': 'MOVIMIENTO EN MASA',
#     'DELIZAMIENTO': 'MOVIMIENTO EN MASA',
#     'DESLIZAMIENTO': 'MOVIMIENTO EN MASA',
#     'ERUPCION': 'ERUPCIÓN VOLCÁNICA',
#     'ERUPCIÓN VOLCANICA': 'ERUPCIÓN VOLCÁNICA',
#     'CONTAMINACION': 'CONTAMINACION AMBIENTAL',
#     'MAR DELEVA': 'MAR DE LEVA',
#     'ACCIDENTE DE TRANSITO': 'ACCIDENTE TRANSPORTE TERRESTRE',
#     'ACCIDENTE TERRESTRE': 'ACCIDENTE TRANSPORTE TERRESTRE',
#     'INCENDIO DE RESIDUO VEGETAL': 'INCENDIO DE COBERTURA VEGETAL',
#     'ACCIDENTE FLUVIAL': 'ACCIDENTE TRANSPORTE MARITIMO O FLUVIAL',
#     'ACCIDENTE MARITIMO': 'ACCIDENTE TRANSPORTE MARITIMO O FLUVIAL'
# }

# # Reemplazar los valores en la columna "TIPO DE EVENTO" según el diccionario
# df4['TIPO DE EVENTO'] = df4['TIPO DE EVENTO'].replace(reemplazos_unificacion)


In [40]:
#Guardo el df como procesamiento de los datos de tipos de evento

# df4.to_csv('Rtado_prueba/df_TiposEvento.csv',index=False)
# df4.to_excel('Rtado_prueba/df_TiposEvento.xlsx',index=False)

In [41]:
# #Filtro los tipos de evento con los que voy a trabajar originalmente que son :

# #  'VENDAVAL' 'MOVIMIENTO EN MASA' 'INUNDACION' 'TORMENTA ELECTRICA' 'CRECIENTE SUBITA' 'AVENIDA TORRENCIAL'

# #Creo una lista con los eventos:

# eventos_filtro=['VENDAVAL' ,'MOVIMIENTO EN MASA', 'INUNDACION','TORMENTA ELECTRICA', 'CRECIENTE SUBITA', 'AVENIDA TORRENCIAL']

# #se guardan los eventos de interés, en las fechas que se tienen (2001 a 2023)

# df4_filtro=df4[df4['TIPO DE EVENTO'].isin(eventos_filtro)]

# # Crear una copia explícita del DataFrame para evitar la advertencia
# df4_filtro = df4_filtro.copy()

# # Convertir la columna "FECHA" a formato de fecha
# df4_filtro['FECHA'] = pd.to_datetime(df4_filtro['FECHA'], errors='coerce')

In [42]:
#Selecciono el periodo que quiero trabajar, en este caso de 2001 a 2021
# # Asegurarse de que la columna "FECHA" esté en formato de fecha
# # Convertir la columna "FECHA" al formato de fecha, convirtiendo fechas no válidas a NaT

# # Crear una copia del DataFrame original con el periodo de interés
# df4_periodo2021 = df4_filtro[(df4_filtro['FECHA'].dt.year >= 2001) & (df4_filtro['FECHA'].dt.year <= 2021)].copy()


# # Convertir la columna "FECHA" a formato de fecha
# df4_periodo2021['FECHA'] = pd.to_datetime(df4_filtro['FECHA'], errors='coerce')





In [43]:
# #Para tener como alternativa también se guarda el dataframe que contiene los eventos de interés de 2001 a 2023

# # Crear una copia del DataFrame original
# df4_periodo2023 = df4_filtro[(df4_filtro['FECHA'].dt.year >= 2001) & (df4_filtro['FECHA'].dt.year <= 2023)].copy()

# # Convertir la columna "FECHA" a formato de fecha
# df4_periodo2023['FECHA'] = pd.to_datetime(df4_filtro['FECHA'], errors='coerce')

# # Eliminar las filas donde la columna "FECHA" es NaT
# df4_periodo2023 = df4_periodo2023.dropna(subset=['FECHA'])

In [44]:
#GUARDAMOS LOS DATAFRAMES

# df4_periodo2021.to_csv('Rtado_prueba/desastres_2001_2021.csv',index=False)

# df4_periodo2023.to_csv('Rtado_prueba/desastres_2001_2023.csv',index=False)


In [45]:
# #Me faltó revisar que todos los códigos de municipios estuvieran completos
# #Vuelvo a importar el archivo

# df4_2023=pd.read_csv("Rtado_prueba/desastres_2001_2023.csv", encoding='UTF-8')



In [46]:
# #Busco datos vacíos en la columna CÓDIGO DANE DEL MUNICIPIO
# # Filtrar las filas donde "CÓDIGO DANE DEL MUNICIPIO" está vacío o es nulo
# filas_vacias = df4_2023[pd.isna(df4_2023['CÓDIGO DANE DEL MUNICIPIO'])]

# filas_vacias

In [47]:
# #municipios que tienen celdas de códigos como Nans:

# mpios_vacios=filas_vacias['MUNICIPIO'].unique()
# mpios_vacios

In [48]:
# #Reemplazo los valores del CÓDIGO DEL MUNICIPIO DANE NaNs por los correspondientes apoyandome en los códigos del DANE oficiales


# # Lista de tuplas con el formato (Municipio, Departamento, Código DANE)
# actualizaciones = [
#     ('SAN LUIS DE PALENQUE', 'CASANARE', 85.325),
#     ('EL PEÑOL', 'NARIÑO', 52.254),
#     ('BUENAVISTA', 'CÓRDOBA', 23.079),
#     ('BUENAVISTA', 'BOYACÁ', 15.109),
#     ('BUENAVISTA', 'QUINDÍO', 63.111),
#     ('BUENAVISTA','SUCRE',70.110),
#     ('SAN PABLO', 'BOLÍVAR', 13.670),
#     ('SAN PABLO', 'NARIÑO', 52.693)
# ]

# # Recorrer la lista y aplicar los cambios
# for municipio, departamento, codigo in actualizaciones:
#     df4_2023.loc[(df4_2023['MUNICIPIO'] == municipio) & 
#                         (df4_2023['DEPARTAMENTO'] == departamento), 
#                         'CÓDIGO DANE DEL MUNICIPIO'] = codigo



In [49]:
# #Se crea una columna con los códigos del departamento

# # Crear una nueva columna "COD_DEP"  que serían los códigos de departamento.
#    #Uso la columna de código dane del municipio porque el número entero es el código del departamento
# df4_2023['COD_DEP'] = np.floor(df4_2023['CÓDIGO DANE DEL MUNICIPIO']).astype(int)

# #Y convierto los números a str para unificarlos con los códigos de shapefiles del DANE
# # Convertir los valores de "COD_DEP" a cadenas de texto y asegurarse de que tengan 2 dígitos
# df4_2023['COD_DEP'] = df4_2023['COD_DEP'].astype(str).str.zfill(2)

In [50]:
# #También creo una nueva columna del código del municipio sin el cod departamento para unificar formatos respecto a shapefiles DANE

# # Crear la nueva columna "COD_MUN" extrayendo la parte después del punto
# df4_2023['COD_MUN'] = df4_2023['CÓDIGO DANE DEL MUNICIPIO'].astype(str).str.split('.').str[1]

# # Asegurarse de que todos los códigos tengan 3 dígitos, agregando ceros a la derecha si es necesario
# df4_2023['COD_MUN'] = df4_2023['COD_MUN'].apply(lambda x: x.ljust(3, '0'))



In [51]:

#  # Crear una copia del DataFrame original con el periodo de interés
    
#     #primero lo convierto a datetime a FECHA
# #Convertir la columna "FECHA" a formato de fecha
# df4_2023['FECHA'] = pd.to_datetime(df4_2023['FECHA'], errors='coerce')    

# #Creo copia del dataframe para el periodo 2008-2020
# df4_2021 = df4_2023[(df4_2023['FECHA'].dt.year >= 2001) & (df4_2023['FECHA'].dt.year <= 2021)].copy()

In [52]:
# #GUARDAMOS LOS DATAFRAMES

# df4_2020.to_csv('Rtado_prueba/desastres_2001_2021.csv',index=False)
# df4_2020.to_excel('Rtado_prueba/desastres_2001_2021.xlsx',index=False)
# df4_2023.to_csv('Rtado_prueba/desastres_2001_2023.csv',index=False)
# df4_2023.to_excel('Rtado_prueba/desastres_2001_2023.xlsx',index=False)

In [53]:
# #Llamo el df creado anteriormente (df4_2023 y df4_2020)
#  #llamo también los datos del shapefile de Mpios_Col_DANE_2023
    
# df5_2023=pd.read_csv("Rtado_prueba/desastres_2001_2023.csv", encoding='UTF-8')
# DANE=pd.read_csv("D:/Tesis_qgis/P_Col/Mpios_Col_DANE_2023.csv", encoding='UTF-8')

# #La idea es observar diferencias entre ellos para Unificar nombres/códigos y poder realizar las gráficas

In [54]:
#Configuro la información para que tengan el mismo formato de códigos de municipio y departamento en ambos dfs

# #Primero lo configuro en df5_2023 (información de desastres)------

# #Configuro la info de DEPARTAMENTOS
# # # Convertir los valores de "COD_DEP" a cadenas de texto y asegurarse de que tengan 2 dígitos
# df5_2023['COD_DEP'] = df5_2023['COD_DEP'].astype(str).str.zfill(2)

# #Configuro la info de MUNICIPIOS
# # # Convertir los valores de "COD_MUN" a cadenas de texto y asegurarse de que tengan 3 dígitos
# df5_2023['COD_MUN'] = df5_2023['COD_MUN'].astype(str).str.zfill(3)

# #Creo una nueva columna que junte la info de COD_DEP y COD_MUN para tenerla en el mismo formato que DANE

# df5_2023['COD_DEP_MUN']= df5_2023['COD_DEP'] + df5_2023['COD_MUN']

In [55]:
# #Ahora configuro la información para DANE

# #Configuro la info de DEPARTAMENTOS
# # # Convertir los valores de "dpto_ccdgo" a cadenas de texto y asegurarse de que tengan 2 dígitos
# DANE['dpto_ccdgo'] = DANE['dpto_ccdgo'].astype(str).str.zfill(2)

# #Configuro la info de MUNICIPIOS
# # # Convertir los valores de "dpto_ccdgo" a cadenas de texto y asegurarse de que tengan 3 dígitos
# DANE['mpio_ccdgo'] = DANE['mpio_ccdgo'].astype(str).str.zfill(3)

# #Configuro la info de DEPTOS + MPIOS (mpio_cdpmp)
# DANE['mpio_cdpmp'] = DANE['mpio_cdpmp'].astype(str).str.zfill(5)

In [56]:
# #Comparar las columnas de mpio_cdpmp en DANE y COD_DEP_MUN para ver diferencias

# # Obtener los códigos únicos de la columna "COD_DEP_MUN" en df5_2023
# codigos_df5_2023 = df5_2023['COD_DEP_MUN'].unique()

# # Obtener los códigos únicos de la columna "mpio_cdpmp" en DANE
# codigos_dane = DANE['mpio_cdpmp'].unique()

# # Convertir los arrays a conjuntos para facilitar la comparación
# set_df5_2023 = set(codigos_df5_2023)
# set_dane = set(codigos_dane)

# # Encontrar los códigos que están en df5_2023 pero no en DANE
# codigos_faltantes_en_dane = set_df5_2023 - set_dane

# # Encontrar los códigos que están en DANE pero no en df5_2023
# codigos_faltantes_en_df5_2023 = set_dane - set_df5_2023

# # Mostrar resultados
# print("Códigos que están en df5_2023 pero no en DANE:", codigos_faltantes_en_dane)
# print("Códigos que están en DANE pero no en df5_2023:", codigos_faltantes_en_df5_2023)



Códigos que están en df5_2023 pero no en DANE: {'27086'}
Códigos que están en DANE pero no en df5_2023: {'94888', '94885', '91669', '94883', '97511', '05664', '94887', '91460', '94886', '97777', '94884', '91430', '94343', '97666'}


In [67]:
#Solo hay un municipio que está en desastres_2023 pero no en DANE y es BELÉN DE BAJIRÁ, que, no es mpio es corregimiento(o era hasta 2024, ya sí es mpio)
#  #Los registros de BELÉN DE BAJIRÁ se anexan a los registros del municipio de riosucio en el CHOCÓ
#   #COD_DEP_MUN de RIOSUCIO es: 27615 y el de BELÉN DE BAJIRÁ es 27086
# belen_bajira= df5_2023[df5_2023['COD_DEP_MUN']=='27086']
# belen_bajira

,FECHA,DEPARTAMENTO,MUNICIPIO,TIPO DE EVENTO,DIVIPOLA,FALLECIDOS,HERIDOS,DESAPARECIDOS,AFECTADOS,FAMILIAS,VIVIENDAS DESTRUIDAS,VIVIENDAS AVERIADAS,CÓDIGO DANE DEL MUNICIPIO,COD_DEP,COD_MUN,COD_DEP_MUN
33,2001-04-19,CHOCÓ,BELÉN DE BAJIRA,VENDAVAL,0,0,0,0,180,36,0,36,27.086,27,086,27086
506,2002-08-05,CHOCÓ,BELÉN DE BAJIRA,VENDAVAL,0,0,0,0,640,128,0,87,27.086,27,086,27086
1994,2005-05-19,CHOCÓ,BELÉN DE BAJIRA,INUNDACION,0,0,0,0,"4,600",920,0,0,27.086,27,086,27086
2001,2005-05-19,CHOCÓ,BELÉN DE BAJIRA,INUNDACION,0,0,0,0,0,0,0,0,27.086,27,086,27086
31105,2022-11-21,CHOCÓ,BELÉN DE BAJIRA,INUNDACION,27086,0,0,0,"7,912","1,978",0,894,27.086,27,086,27086


In [58]:
# #Guardo dataframe para el periodo 2001-2021

# # #Convertir la columna "FECHA" a formato de fecha
# df5_2023['FECHA'] = pd.to_datetime(df5_2023['FECHA'], errors='coerce')    


# # #Creo copia del dataframe para el periodo 2008-2020
# df5_2021 = df5_2023[(df5_2023['FECHA'].dt.year >= 2001) & (df5_2023['FECHA'].dt.year <= 2021)].copy()

In [59]:
# # #GUARDAMOS LOS DATAFRAMES

# df5_2021.to_csv('Rtado_prueba/desastres_2001_2021_graf.csv',index=False)
# df5_2023.to_csv('Rtado_prueba/desastres_2001_2023_graf.csv',index=False)
